In [ ]:
# Data Preparation Notebook

## Objectives
- Clean the data by removing outliers
- Handle any data quality issues  
- Create new features to improve predictions
- Prepare data for machine learning

## Inputs
- Raw data from inputs/datasets/collection/uk_housing_small.csv

## Outputs
- Cleaned dataset ready for modeling
- New engineered features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for nice plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)